## 그룹화(Group) 실습

- DF/SR.groupby() 메서드로 생성
- 그룹 단위로 평균, 최대, 최소 등과 같은 집계 메서드 적용 가능

In [1]:
# 모듈 로딩
import pandas as pd
import numpy as np

df = pd.DataFrame(
    [
        ("bird", "Falconiformes", 389.0),
        ("bird", "Psittaciformes", 24.0),
        ("mammal", "Carnivora", 80.2),
        ("mammal", "Primates", np.nan),
        ("mammal", "Carnivora", 58),
    ],
    index=["falcon", "parrot", "lion", "monkey", "leopard"],
    columns=("class", "order", "max_speed"),
)
df

,class,order,max_speed
falcon,bird,Falconiformes,389.0
parrot,bird,Psittaciformes,24.0
lion,mammal,Carnivora,80.2
monkey,mammal,Primates,NaN
leopard,mammal,Carnivora,58.0


In [6]:
print(f'column => {df.columns} \nindex => {df.index}', sep='\n')

column => Index(['class', 'order', 'max_speed'], dtype='object') 
index => Index(['falcon', 'parrot', 'lion', 'monkey', 'leopard'], dtype='object')


In [7]:
# 그룹 만들기 --> 객체변수명.groupby(컬럼명)
classGroup=df.groupby(by='class')
classGroup

In [8]:
classGroup.groups

{'bird': ['falcon', 'parrot'], 'mammal': ['lion', 'monkey', 'leopard']}

In [11]:
# 반복이 가능한 객체 => for~in 사용 가능
for group in classGroup:
    print(group)

('bird',        class           order  max_speed
falcon  bird   Falconiformes      389.0
parrot  bird  Psittaciformes       24.0)
('mammal',           class      order  max_speed
lion     mammal  Carnivora       80.2
monkey   mammal   Primates        NaN
leopard  mammal  Carnivora       58.0)


In [16]:
for key, group in classGroup:
    print(f'KEY --- \n{key}')
    print(f'GROUP --- \n{group}\n')

KEY --- 
bird
GROUP --- 
       class           order  max_speed
falcon  bird   Falconiformes      389.0
parrot  bird  Psittaciformes       24.0

KEY --- 
mammal
GROUP --- 
          class      order  max_speed
lion     mammal  Carnivora       80.2
monkey   mammal   Primates        NaN
leopard  mammal  Carnivora       58.0



In [12]:
# GroupBy 객체 속성 살펴보기
# 그룹키 : [요소 인덱스, ...]
classGroup.groups

{'bird': ['falcon', 'parrot'], 'mammal': ['lion', 'monkey', 'leopard']}

In [13]:
# 특정 그룹키에 해당하는 데이터만 추출 => get_group(키)
classGroup.get_group('bird')

,class,order,max_speed
falcon,bird,Falconiformes,389.0
parrot,bird,Psittaciformes,24.0


In [18]:
# 그룹 전체에 대한 메서드 적용
classGroup.describe(include='all')

order                                                           ...  \
       count unique            top freq mean  std  min  25%  50%  75%  ...   
class                                                                  ...   
bird       2      2  Falconiformes    1  NaN  NaN  NaN  NaN  NaN  NaN  ...   
mammal     3      2      Carnivora    2  NaN  NaN  NaN  NaN  NaN  NaN  ...   

       max_speed                                                           \
          unique top freq   mean         std   min     25%    50%     75%   
class                                                                       
bird         NaN NaN  NaN  206.5  258.093975  24.0  115.25  206.5  297.75   
mammal       NaN NaN  NaN   69.1   15.697771  58.0   63.55   69.1   74.65   

               
          max  
class          
bird    389.0  
mammal   80.2  

[2 rows x 22 columns]

In [19]:
classGroup.std()

,max_speed
class,
bird,258.093975
mammal,15.697771


In [22]:
# 특정 그룹에 대한 것만 처리 => get_group(키)
birdGroup=classGroup.get_group('bird')

In [23]:
birdGroup.describe()

,max_speed
count,2.000000
mean,206.500000
std,258.093975
min,24.000000
25%,115.250000
50%,206.500000
75%,297.750000
max,389.000000


In [24]:
birdGroup.max(), birdGroup.min()

(class                  bird
 order        Psittaciformes
 max_speed             389.0
 dtype: object,
 class                 bird
 order        Falconiformes
 max_speed             24.0
 dtype: object)

In [25]:
# 집계함수를 한꺼번에 적용 => agg([함수명, 함수명, ...], axis=0)
birdGroup.agg(['min', 'max'])

,class,order,max_speed
min,bird,Falconiformes,24.0
max,bird,Psittaciformes,389.0


In [29]:
# 집계함수를 한꺼번에 적용 => agg({컬럼명:함수명, 컬럼명:함수명, ...}, axis=0)
birdGroup.agg({'order':'max', 'max_speed':['min', 'sum']})

,order,max_speed
max,Psittaciformes,NaN
min,NaN,24.0
sum,NaN,413.0


In [31]:
# 사용자 정의 함수
def getMaxMin(x):
    print(x)
    return x.max()-x.min()

birdGroup.agg({'order':'max', 'max_speed':[getMaxMin]})

389.0
falcon    389.0
parrot     24.0
Name: max_speed, dtype: float64


,order,max_speed
max,Psittaciformes,NaN
getMaxMin,NaN,365.0


In [33]:
# 그룹에서 필터링하기 ----> filter(True/False)
birdGroup.filter(items=['max_speed'])

,max_speed
falcon,389.0
parrot,24.0


In [35]:
birdGroup['max_speed']>=300

falcon     True
parrot    False
Name: max_speed, dtype: bool

In [38]:
birdGroup[birdGroup['max_speed']>=300]

,class,order,max_speed
falcon,bird,Falconiformes,389.0


In [44]:
# 필터는... 모르겠다
#birdGroup.filter(items='max_speed', lambda x:x>=300)